In [6]:
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re
import cx_Oracle
from psycopg2 import sql


# PostgreSQL connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

# Oracle connection parameters
oracle_username = 'apps'
oracle_password = 'apps'
oracle_host = 'csebsd2db.cswg.com'
oracle_port = '1521'
oracle_service_name = 'csebsd2_int'



post_conn = psycopg2.connect(**db_params)
post_cur = post_conn.cursor()

# XML file path
# xml_file_path = 'D:\\W\\RELEX\\RELEX\\Broker and Vendor\\Inbound Files\\VENDOR.xml'
# xml_file_path = 'D:\\W\\RELEX\\RELEX\\Broker and Vendor\\Inbound Files\\BROKER.xml'
text_data_path = 'D:\\W\\RELEX\\RELEX\\Label Flag\AWIROBLABEL.IN645.2023022804100111'


# Open the XML file
with open(text_data_path, 'r', encoding='utf-8') as file:
    text_data = file.read()
try:
    for line in text_data.split('\n'):
        try:
            print(line)
        except Exception as e:
            print(f"Error processing line: {line}. Exception: {e}")
except Exception as e:
    print(f"Error processing file: {file}. Error: {str(e)}")



0101AP00197306202302270730Y01         000085545N                              S/W TO FOLLOW AT LEADTIME                                   001178T097                                                                                            
0101AP00198572202302280330Y01         000085541N                              S/W 94198572,9419881          SHIPPER 4/3 S/W 94198572,941  999999B101                                                                                            
0101AP00198881202302280330Y01         000085540N                                                            S/W 94198572,9419881          999999B101                                                                                            
0102AP00198910202302280330Y01         000085553N                              2/21                                                          SMITT033                                                                                            
0101AP00198937202302270730Y01       

In [16]:
text_data_path = 'D:\\W\\RELEX\\RELEX\\Label Flag\AWIROBLABEL.IN645.2023022804100111'

# Define positions for slicing each line into tuples
[(0, 2), (2, 4), (4, 6), (6, 8), (8, 14), (14, 16), (16, 18), (18, 20), (20, 22), (22, 24), (24, 26), (26, 27), (27, 29), (29, 30), (30, 32), (32, 34), (34, 36), (36, 38), (38, 47), (47, 48), (48, 78), (78, 138), (138, 144), (144, 145), (145, 152), (152, 163), (163, 171), (171, 175), (175, 176), (176, 240)]

# Open the file and read its content
with open(text_data_path, 'r', encoding='utf-8') as file:
    text_data = file.read()
def process_text_data(text_data, positions):
    # List to store extracted data as tuples
    extracted_data = []

    try:
        for line in text_data.split('\n'):
            try:
                # Slice each line based on positions and create a tuple
                line_data = tuple(line[start:end].strip() for start, end in positions)

                # Check if the tuple is not empty (contains at least one non-blank element)
                if any(field for field in line_data):
                    # Append the tuple to the list
                    extracted_data.append(line_data)

            except Exception as e:
                print(f"Error processing line: {line}. Exception: {e}")

    except Exception as e:
        print(f"Error processing text data. Error: {str(e)}")

    # Return the list of tuples
    return extracted_data


result = process_text_data(text_data, positions)
print(result)




[('01', '01', 'AP', '00', '197306', '20', '23', '02', '27', '07', '30', 'Y', '01', '', '', '', '', '', '000085545', 'N', '', 'S/W TO FOLLOW AT LEADTIME', '001178', 'T', '097', '', '', '', '', ''), ('01', '01', 'AP', '00', '198572', '20', '23', '02', '28', '03', '30', 'Y', '01', '', '', '', '', '', '000085541', 'N', '', 'S/W 94198572,9419881          SHIPPER 4/3 S/W 94198572,941', '999999', 'B', '101', '', '', '', '', ''), ('01', '01', 'AP', '00', '198881', '20', '23', '02', '28', '03', '30', 'Y', '01', '', '', '', '', '', '000085540', 'N', '', 'S/W 94198572,9419881', '999999', 'B', '101', '', '', '', '', ''), ('01', '02', 'AP', '00', '198910', '20', '23', '02', '28', '03', '30', 'Y', '01', '', '', '', '', '', '000085553', 'N', '', '2/21', 'SMIT', 'T', '033', '', '', '', '', ''), ('01', '01', 'AP', '00', '198937', '20', '23', '02', '27', '07', '30', 'Y', '01', '', '', '', '', '', '000085544', 'N', '', '', '001178', 'T', '097', '', '', '', '', ''), ('01', '03', 'AP', '00', '198988', '20'

In [15]:
# Define positions for slicing each line into tuples
lengths = [
    2, 2, 2, 2, 6, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 9, 1, 30, 60, 6, 1, 7, 11, 8, 4, 1, 64
]

# Calculate the end positions based on the lengths
end_positions = [sum(lengths[:i + 1]) for i in range(len(lengths))]

# Create positions as tuples of start and end positions
positions = [(start, end) for start, end in zip([0] + end_positions, end_positions)]

print(positions)


[(0, 2), (2, 4), (4, 6), (6, 8), (8, 14), (14, 16), (16, 18), (18, 20), (20, 22), (22, 24), (24, 26), (26, 27), (27, 29), (29, 30), (30, 32), (32, 34), (34, 36), (36, 38), (38, 47), (47, 48), (48, 78), (78, 138), (138, 144), (144, 145), (145, 152), (152, 163), (163, 171), (171, 175), (175, 176), (176, 240)]


In [18]:
def insert_data_to_table(extracted_data, table_name, post_cur):
    # Define the columns present in the table
    table_columns = [
        "FACILITY", "WARE_HOUSE", "RECORD_TYPE", "FILLER", "PO_NUMBER", "APPT_CENTURY", "APPT_YEAR", "APPT_MONTH",
        "APPT_DATE", "APPT_TIME_HH", "APPT_TIME_MIN", "PALLET_LABEL_PRINT", "PARENT_FACILITY", "PO_RE_ROUTE_FLAG",
        "TO_FACILITY", "TO_WAREHOUSE", "FROM_FACILITY", "FROM_WAREHOUSE", "RECEIPT_NUMBER", "APPT_CANCEL_FLAG",
        "WHSE_COMMENTS", "WHSE_COMMENTS2", "CARRIER", "TRANSPORT_TYPE", "RECEIPT_SLOT", "APPOINTMENT_ID",
        "DEPARTURE_DATE", "DEPARTURE_TIME", "LABEL_DEL_EXAMPT_FLAG", "FILLER1"
    ]

    # Filter out columns that are not present in the data
    valid_columns = [col for col in table_columns if any(data[col] for data in extracted_data)]

    # Generate the column names dynamically based on the valid columns
    columns = ", ".join(valid_columns)

    # Generate the placeholders for values dynamically
    value_placeholders = ", ".join(["%s"] * len(valid_columns))

    # Create the INSERT query
    insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({value_placeholders})"

    # Prepare the data as a list of tuples, considering only valid columns
    data_to_insert = [[data[col] for col in valid_columns] for data in extracted_data]

    try:
        # Execute the INSERT query with the data
        post_cur.executemany(insert_query, data_to_insert)

        # Commit the changes
        post_cur.commit()

        print("Data inserted successfully.")

    except Exception as e:
        print(f"Error inserting data to table: {table_name}. Exception: {e}")
        post_cur.rollback()

# Example usage
text_data_path = 'D:\\W\\RELEX\\RELEX\\Label Flag\\AWIROBLABEL.IN645.2023022804100111'
table_name = 'CSPOMS.XXPO_LABEL_FLAG_AWI_TRICEPS_STG'

# Extract data from text file
# extracted_data = process_text_data(text_data_path)

# Insert data into the table
insert_data_to_table(result, table_name, post_cur)


TypeError: tuple indices must be integers or slices, not str

In [20]:
def insert_data_to_table(extracted_data, table_name, post_cur, column_names):
    # Create the INSERT query with explicit column names and placeholders
    insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s']*len(column_names))})"
    print(insert_query)
    try:
        # Execute the INSERT query with the data
        post_cur.executemany(insert_query, extracted_data)

        # Commit the changes
        post_conn.commit()

        print("Data inserted successfully.")

    except Exception as e:
        print(f"Error inserting data to table: {table_name}. Exception: {e}")
#         post_cur.rollback()

# Example usage
text_data_path = 'D:\\W\\RELEX\\RELEX\\Label Flag\\AWIROBLABEL.IN645.2023022804100111'
table_name = 'CSPOMS.XXPO_LABEL_FLAG_AWI_TRICEPS_STG'

# Column names in the same order as your data fields
column_names = ['FACILITY', 'WARE_HOUSE', 'RECORD_TYPE', 'FILLER', 'PO_NUMBER', 'APPT_CENTURY', 'APPT_YEAR',
                'APPT_MONTH', 'APPT_DATE', 'APPT_TIME_HH', 'APPT_TIME_MIN', 'PALLET_LABEL_PRINT',
                'PARENT_FACILITY', 'PO_RE_ROUTE_FLAG', 'TO_FACILITY', 'TO_WAREHOUSE', 'FROM_FACILITY',
                'FROM_WAREHOUSE', 'RECEIPT_NUMBER', 'APPT_CANCEL_FLAG', 'WHSE_COMMENTS', 'WHSE_COMMENTS2',
                'CARRIER', 'TRANSPORT_TYPE', 'RECEIPT_SLOT', 'APPOINTMENT_ID', 'DEPARTURE_DATE', 'DEPARTURE_TIME',
                'LABEL_DEL_EXAMPT_FLAG', 'FILLER1']

# # Extract data from text file
# extracted_data = process_text_data(text_data_path)

# Insert data into the table
insert_data_to_table(result, table_name, post_cur, column_names)


INSERT INTO CSPOMS.XXPO_LABEL_FLAG_AWI_TRICEPS_STG (FACILITY, WARE_HOUSE, RECORD_TYPE, FILLER, PO_NUMBER, APPT_CENTURY, APPT_YEAR, APPT_MONTH, APPT_DATE, APPT_TIME_HH, APPT_TIME_MIN, PALLET_LABEL_PRINT, PARENT_FACILITY, PO_RE_ROUTE_FLAG, TO_FACILITY, TO_WAREHOUSE, FROM_FACILITY, FROM_WAREHOUSE, RECEIPT_NUMBER, APPT_CANCEL_FLAG, WHSE_COMMENTS, WHSE_COMMENTS2, CARRIER, TRANSPORT_TYPE, RECEIPT_SLOT, APPOINTMENT_ID, DEPARTURE_DATE, DEPARTURE_TIME, LABEL_DEL_EXAMPT_FLAG, FILLER1) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
Data inserted successfully.
